In [ ]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
df = pd.read_csv('tw_tweets_users_media_places.csv')
df.head()

,tweet_id,text,context_annotations_count,count_annotations,count_cashtags,count_hashtags,count_mentions,count_urls,created_at_tweet,lang,...,location,protected,verified,media_type,height,width,preview_image_url,country,name_place,place_type
0,1440484799970304000,This was my grandson this morning (w/autism)! ...,1,0.0,0.0,0.0,0.0,1.0,2021-09-22T01:15:13.000Z,en,...,"Victoria, BC",False,False,photo,405,813,https://pbs.twimg.com/media/E_2hSs4UcAAIOK5.jpg,Canada,Langford,city
1,1439618825171963904,Wow!! Been into #York for the first time since...,2,2.0,0.0,3.0,0.0,1.0,2021-09-19T15:54:09.000Z,en,...,"Hessay, York",False,False,photo,2048,1536,https://pbs.twimg.com/media/E_qNsE1X0AQmoK_.jpg,United Kingdom,Hessay,city
2,1248872872837332992,Sad number of ppl who lost life due to covid-1...,3,0.0,0.0,0.0,0.0,1.0,2020-04-11T07:17:50.000Z,en,...,"Maidstone, South East",False,False,photo,288,278,https://pbs.twimg.com/media/EVTjQcoXsAAlrfq.jpg,United Kingdom,Maidstone,city
3,1250729294051053568,Webinar now available‘Staying healthy at home ...,1,2.0,0.0,3.0,0.0,2.0,2020-04-16T10:14:35.000Z,en,...,"Maidstone, South East",False,False,photo,2048,2048,https://pbs.twimg.com/media/EVt7pYTXkAMGzxj.jpg,United Kingdom,Maidstone,city
4,1249612131433095168,Webinar now available‘Staying healthy at home ...,1,2.0,0.0,3.0,0.0,2.0,2020-04-13T08:15:23.000Z,en,...,"Maidstone, South East",False,False,photo,2048,2048,https://pbs.twimg.com/media/EVeDlp7X0AMuN6X.jpg,United Kingdom,Maidstone,city


In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Word lists and lexicons in nltk: https://www.nltk.org/howto/corpus.html#word-lists-and-lexicons
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("This is a really great tweet!")

# Output:
# [nltk_data] Downloading package vader_lexicon to /root/nltk_data...
# [nltk_data]   Package vader_lexicon is already up-to-date!
# {'compound': 0.6893, 'neg': 0.0, 'neu': 0.461, 'pos': 0.539}

In [ ]:
df_tweets['sentiment_overall'] = 0.0
df_tweets['sentiment_neg'] = 0.0
df_tweets['sentiment_neu'] = 0.0
df_tweets['sentiment_pos'] = 0.0

for row in df_tweets.itertuples():
sentiment = sia.polarity_scores(row[8])
df_tweets.loc[row[0], 'sentiment_overall'] = sentiment['compound']
df_tweets.loc[row[0], 'sentiment_neg'] = sentiment['neg']
df_tweets.loc[row[0], 'sentiment_neu'] = sentiment['neu']
df_tweets.loc[row[0], 'sentiment_pos'] = sentiment['pos']

df_tweets.head()

In [ ]:
import statsmodels.api as sm

df_dummy = pd.get_dummies(df_tweets, columns=['Gender', 'Weekday'], drop_first=True)

y = df_dummy.RetweetCount
X = df_dummy.drop(columns=['text', 'RetweetCount', 'sentiment_overall', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos']).assign(const=1)

print(sm.OLS(y, X).fit().summary())

# Output:
# R-squared: 0.246


In [ ]:
X = df_dummy.drop(columns=['text', 'RetweetCount']).assign(const=1)

print(sm.OLS(y, X).fit().summary())

# Output:
# R-squared: 0.252

#                         coef    std err          t      P>|t|
# -------------------------------------------------------------
# sentiment_overall   -21.5958      7.617     -2.835      0.016
# sentiment_neg     -2564.4798   6662.465     -0.385      0.490
# sentiment_neu     -2529.1936   6659.568     -0.380      0.493
# sentiment_pos     -2487.2074   6659.596     -0.373      0.497
